In [1]:
%%capture
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from scipy import integrate
from scipy import interpolate
import math
from scipy import signal
import pandas as pd
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
%run KalmanFilter.ipynb

In [2]:
# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

In [3]:
import weather
weather.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


In [4]:
cities = weather.cities
cities

['Aalborg', 'Aarhus', 'Esbjerg', 'Odense', 'Roskilde']

In [11]:
%%time
df = weather.load_resampled_data()

CPU times: user 8.99 ms, sys: 20.5 ms, total: 29.5 ms
Wall time: 27.5 ms


In [12]:
df.head()
df.drop(('Esbjerg', 'Pressure'), axis=1, inplace=True)
df.drop(('Roskilde', 'Pressure'), axis=1, inplace=True)

In [14]:
df.values.shape
df.head(1)

Aalborg                                Aarhus           \
                       Temp     Pressure WindSpeed WindDir   Temp Pressure   
DateTime                                                                     
1980-03-01 11:00:00     5.0  1007.766667      10.2   280.0    5.0   1008.3   

                                        Esbjerg                       Odense  \
                    WindSpeed WindDir      Temp  WindSpeed WindDir      Temp   
DateTime                                                                       
1980-03-01 11:00:00      15.4   290.0  6.083333  12.383333   310.0  6.142857   

                                                    Roskilde             \
                        Pressure  WindSpeed WindDir     Temp  WindSpeed   
DateTime                                                                  
1980-03-01 11:00:00  1011.066667  12.585714   290.0      5.0  11.466667   

                             
                    WindDir  
DateTime                     
1980-03-01 11:00:00   280.0

In [15]:
df['Various', 'Day'] = df.index.dayofyear
df['Various', 'Hour'] = df.index.hour

In [16]:
target_city = 'Odense'
target_names = ['Temp', 'WindSpeed', 'Pressure']

In [30]:
shift_days = 1
shift_steps = shift_days * 24  # Number of hours.

In [32]:
df_targets = df[target_city][target_names].shift(-shift_steps)
df[target_city][target_names].head(shift_steps + 5)

,Temp,WindSpeed,Pressure
DateTime,,,
1980-03-01 11:00:00,6.142857,12.585714,1011.066667
1980-03-01 12:00:00,7.000000,11.300000,1011.200000
1980-03-01 13:00:00,7.000000,12.118182,1011.300000
1980-03-01 14:00:00,6.857143,12.742857,1011.400000
1980-03-01 15:00:00,6.000000,12.400000,1011.500000
1980-03-01 16:00:00,4.909091,12.618182,1011.688889
1980-03-01 17:00:00,3.953488,12.646512,1011.877778
1980-03-01 18:00:00,3.674419,11.725581,1012.066667
1980-03-01 19:00:00,3.395349,10.804651,1012.255556


In [33]:
x_data = df.values[0:-shift_steps]
print(type(x_data))
print("Shape:", x_data.shape)

<class 'numpy.ndarray'>
Shape: (333085, 20)


In [34]:
y_data = df_targets.values[:-shift_steps]
print(type(y_data))
print("Shape:", y_data.shape)
num_data = len(x_data)
train_split = 0.9
num_train = int(train_split * num_data)
num_test = num_data - num_train


<class 'numpy.ndarray'>
Shape: (333085, 3)


In [35]:
x_train = x_data[0:num_train]
x_test = x_data[num_train:]
len(x_train) + len(x_test)

333085

In [39]:
num_x_signals = x_data.shape[1]

In [40]:
num_y_signals = y_data.shape[1]
print(x_train)

[[5.00000000e+00 1.00776667e+03 1.02000000e+01 ... 2.80000000e+02
  6.10000000e+01 1.10000000e+01]
 [5.00000000e+00 1.00800000e+03 1.03000000e+01 ... 2.80000000e+02
  6.10000000e+01 1.20000000e+01]
 [5.00000000e+00 1.00806667e+03 9.70000000e+00 ... 2.78333333e+02
  6.10000000e+01 1.30000000e+01]
 ...
 [9.90000000e+00 1.00420000e+03 4.10000000e+00 ... 2.70000000e+02
  1.33000000e+02 0.00000000e+00]
 [9.90000000e+00 1.00430000e+03 4.10000000e+00 ... 2.70000000e+02
  1.33000000e+02 1.00000000e+00]
 [9.50000000e+00 1.00440000e+03 4.10000000e+00 ... 2.80000000e+02
  1.33000000e+02 2.00000000e+00]]


In [38]:
print("Min:", np.min(x_train))
print("Max:", np.max(x_train))

Min: -27.0
Max: 1050.8


In [42]:
x_scaler = MinMaxScaler()
x_train_scaled = x_scaler.fit_transform(x_train)